In [9]:
# Helpers
import numpy as np

# DB 
import psycopg2
from django.conf import settings

# Evaluation
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
connection = psycopg2.connect(user = settings.DATABASES["default"]["USER"],
                                  password = settings.DATABASES["default"]["PASSWORD"],
                                  host = settings.DATABASES["default"]["HOST"],
                                  port = settings.DATABASES["default"]["PORT"],
                                  database = settings.DATABASES["default"]["NAME"])
connection.autocommit=True
cursor = connection.cursor()

In [37]:
%%time
query = "select experiment_id, q_matrix, q_matrix_hat, X, concepts from EDM2020_2020_02_19 where method = 'fdtf'"
cursor.execute(query)
row = cursor.fetchone()

CPU times: user 1.69 s, sys: 923 ms, total: 2.62 s
Wall time: 1min 14s


In [38]:
%%time
update_queries = []
total = 0
while row:
    exp_id, q_matrix, q_matrix_hat, X, concepts = row
    
    # Get deleted questions
    q_matrix = np.asarray(q_matrix)
    if concepts != 14:
        attempts_per_question = np.asarray(X).sum(axis=0).sum(axis=1)
        del_questions = [idx for idx,value in enumerate(attempts_per_question) if value < 3]
        q_matrix = np.delete(q_matrix, del_questions, axis=1)

    # Normalize q-matrix
    question_similarity = cosine_similarity(q_matrix.T)
    question_hat_similarity = cosine_similarity(np.asarray(q_matrix_hat).T)
    error = question_similarity - question_hat_similarity
#     error_sum = np.sqrt(np.sum(np.power(error,2)))
    rmse = np.sqrt(np.mean(np.power(error,2)))
    
    update_query = "update EDM2020_2020_02_19 set q_matrix_rmse = %s where experiment_id = %s"
    query_values = (rmse, exp_id)
    query = cursor.mogrify(update_query, query_values)
    update_queries.append(query)
#     cursor.execute(query)
    total += 1
    try:
        row = cursor.fetchone()
    except psycopg2.ProgrammingError:
        row = False

CPU times: user 35.6 s, sys: 137 ms, total: 35.7 s
Wall time: 35.7 s


In [39]:
for query in update_queries:
    cursor.execute(query)

In [40]:
total

406